In [9]:
import numpy as np

In [2]:
class MulLayer:
  """곱셈 계층
  """
  def __init__(self):
    self.x = None
    self.y = None
  
  def forward(self, x, y):
    self.x = x
    self.y = y
    return x * y
  
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x
    return dx, dy

In [3]:
"""
곱셉 계층의 순전파, 역전파 테스트
"""

apple = 100
apple_num = 2
tax_rate = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax_rate)
print(price)

# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dprice, dapple_price, dtax, dapple, dapple_num)

220.00000000000003
1 1.1 200 2.2 110.00000000000001


In [4]:
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    return x + y
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [5]:
"""
곱셈, 덧셈 계층의 순전파, 역전파 테스트
"""

apple_price = 100
apple_num = 2
orange_price = 150
orange_num = 3
tax_rate = 1.1

# 계층들
mul_apple_price = MulLayer()
mul_orange_price = MulLayer()
add_apple_orange_price = AddLayer()
mul_tax = MulLayer()

# 순전파
apple_total_price = mul_apple_price.forward(apple_price, apple_num)
orange_total_price = mul_orange_price.forward(orange_price, orange_num)
total_price = add_apple_orange_price.forward(apple_total_price, orange_total_price)
final_price = mul_tax.forward(total_price, tax_rate)
print("apple_price:", apple_price)
print("apple_num:", apple_num)
print("orange_price:", orange_price)
print("orange_num:", orange_num)
print("tax_rate:", tax_rate)
print("apple_total_price:", apple_total_price)
print("orange_total_price:", orange_total_price)
print("total_price:", total_price)
print("final_price:", final_price)

# 역전파
d_final_price = 1
d_total_price, d_tax = mul_tax.backward(d_final_price)
d_apple_total_price, d_orange_total_price = add_apple_orange_price.backward(d_total_price)
d_apple_price, d_apple_num = mul_apple_price.backward(d_apple_total_price)
d_orange_price, d_orange_num = mul_orange_price.backward(d_orange_total_price)
print("d_final_price:", d_final_price)
print("d_total_price:", d_total_price)
print("d_tax:", d_tax)
print("d_apple_total_price:", d_apple_total_price)
print("d_orange_total_price:", d_orange_total_price)
print("d_apple_price:", d_apple_price)
print("d_apple_num:", d_apple_num)
print("d_orange_price:", d_orange_price)
print("d_orange_num:", d_orange_num)


apple_price: 100
apple_num: 2
orange_price: 150
orange_num: 3
tax_rate: 1.1
apple_total_price: 200
orange_total_price: 450
total_price: 650
final_price: 715.0000000000001
d_final_price: 1
d_total_price: 1.1
d_tax: 650
d_apple_total_price: 1.1
d_orange_total_price: 1.1
d_apple_price: 2.2
d_apple_num: 110.00000000000001
d_orange_price: 3.3000000000000003
d_orange_num: 165.0


In [14]:
class ReluLayer:
  """ReLU 계층
  """

  def __init__(self):
    self.mask = None
  
  def forward(self, x):
    self.mask = (x <= 0)
    out = x.copy()
    out[self.mask] = 0
    return out
  
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

a = np.array([-0.2, -0.1, 0.0, 0.1, 0.2])
dout = np.array([1.0, 1.0, 1.0, 1.0, 1.0])
rl = ReluLayer()
print("a:", a)
print("rl.forward(a):", rl.forward(a))
print("r1.backward(a):", rl.backward(dout))

a: [-0.2 -0.1  0.   0.1  0.2]
rl.forward(a): [0.  0.  0.  0.1 0.2]
r1.backward(a): [0. 0. 0. 1. 1.]


In [20]:
class SigmoidLayer:
  """Sigmoid 계층
  """
  def __init__(self):
    self.out = None

  def forward(self, x):
    """순전파
    순전파의 출력을 저장해 두고 역전파 때 사용한다.
    """
    self.out = 1 / (1 + np.exp(-x))
    return self.out
  
  def backward(self, dout):
    return dout * self.out * (1-self.out)

a = np.array([-10.0, -5.0, 0.0, 5.0, 10.0])
dout = np.array([1.0, 1.0, 1.0, 1.0, 1.0])
sl = SigmoidLayer()
print("a:", a)
print("sl.forward(a):", sl.forward(a))
print("s1.backward(a):", sl.backward(dout))

a: [-10.  -5.   0.   5.  10.]
sl.forward(a): [4.53978687e-05 6.69285092e-03 5.00000000e-01 9.93307149e-01
 9.99954602e-01]
s1.backward(a): [4.53958077e-05 6.64805667e-03 2.50000000e-01 6.64805667e-03
 4.53958077e-05]


In [25]:
class AffineLayer:
  """Affine 계층
  """
  def __init__(self):
    self.X = None
    self.W = None
    self.B = None
    self.dX = None
    self.dW = None
    self.dB = None
    pass

  def forward(self, X, W, B):
    """순전파
    """
    self.X = X
    self.W = W
    self.B = B
    return np.dot(X, W) + B
  
  def backward(self, dout):
    """역전파
    """
    self.dX = np.dot(dout, self.W.T)
    self.dW = np.dot(self.X.T, dout)
    self.dB = np.sum(dout, axis=0)
    return self.dX, self.dW, self.dB

A = np.array([[1,2,3], [4,5,6]])
A.T
print("A:", A)
print("A.T:" , A.T)

X = np.array([[1,2,3], [4,5,6]])
W = np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]])
B = np.array([0.1, 0.2, 0.3])
D = np.array([[1.0, 1.0, 1.0], [1.0, 1.0, 1.0]])
al = AffineLayer()
al_forward_out = al.forward(X, W, B)
al_backward_out = al.backward(D)
print("X:", X)
print("W:", W)
print("B:", B)
print("D:", D)




A: [[1 2 3]
 [4 5 6]]
A.T: [[1 4]
 [2 5]
 [3 6]]
